In [39]:
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local-advanced.ipynb

from azureml.core import  Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

from azureml.core.environment import Environment 
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model,InferenceConfig

interactive_auth = InteractiveLoginAuthentication(tenant_id="9ce70869-60db-44fd-abe8-d2767077fc8f")
ws = Workspace.from_config()


In [40]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: cdh-azml-dev-mlw
Azure region: eastus
Subscription id: 320d8d57-c87c-4434-827f-59ee7d86687a
Resource group: csels-cdh-dev


In [41]:
import sklearn
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

print(sklearn.__version__)

1.1.2


In [42]:
model = Model(workspace=ws,name='breast-cancer',version=2)

In [43]:
import os
print(os.getenv("AZUREML_MODEL_DIR"))
#model_path = Model.get_model_path(os.getenv("AZUREML_MODEL_DIR").split('/')[-2])

None


In [44]:
import os

os.makedirs('./aml_outputs/',exist_ok=True)
model.download(target_dir='./aml_outputs/',exist_ok=True)

'aml_outputs/cancer_model.pkl'

In [45]:
%%writefile ./scoring/score.py

import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION)
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'cancer_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)



input_sample = np.array([[0.393845668409139, -0.7457496952627328, -0.22189398515428446, -0.7676382304726264, -0.6999246324662508, -0.17745016145311038, 
-0.81484548808207, -0.770581761525873, -0.7189314051409194, 0.07367558407930598, -0.4665409990624931, -0.06425072325003647, -0.667435753669589, 
-0.10209930505592255, -0.2928749021935234, -0.18393920631817678, -0.8204798568998558, -0.6065568523529427, -0.395651155530143, 0.3134950272756633,
 -0.8678658791041564, -0.6897420117050609, -0.5598110994362666, -0.723009965205552, -0.6542518616646612, -0.5936861839456196,
 -0.9574791012332506, -0.9270503937797329, -0.7552651134179409, -0.2651797994882257, -1.0640128515011344]])
output_sample = np.array([0])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        print("input_data....")
        print(type(data))
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return "here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

Overwriting ./scripts/score.py


### use custom envirronment

In [46]:
print('scikit-learn=={}'.format(sklearn.__version__))

scikit-learn==1.1.2


In [47]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pip==20.2.4'],
    pip_packages=[
    'azureml-defaults',
    'pandas',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn'
])

#### Create Inference Configuration

* file_path: input parameter to Environment constructor. Manages conda and python package dependencies.
* env.docker.base_dockerfile: any extra steps you want to inject into docker file
* source_directory: holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
* entry_script: contains logic specific to initializing your model and running predictions

In [48]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'breast-cancer-custom-service'

inference_config = InferenceConfig(entry_script='./scripts/score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                memory_gb=1 )
                                                #vnet_name='csels-cdh-dev-vnet', 
                                                #subnet_name='cdh-azml-dev-snet')



service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Failed

Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 1cdd7ede-42c5-4644-ac57-12f9a39e2522
More information can be found using '.get_logs()'
Error:
{
  "code": "IpConfigProfileCannotReferToSubnetInUseWithResource",
  "statusCode": 400,
  "message": "Vnet Service request failed. Reason: IpConfigurationProfile /subscriptions/320d8d57-c87c-4434-827f-59ee7d86687a/resourceGroups/csels-cdh-dev/providers/Microsoft.Network/networkProfiles/aci-network-profile-csels-cdh-dev-vnet-cdh-azml-dev-snet/containerNetworkInterfaceConfigurations/eth0/ipConfigurations/ipconfig1 cannot refer to subnet /subscriptions/320d8d57-c87c-4434-827f-59ee7d86687a/resourceGroups/CSELS-CDH-DEV/providers/Microsoft.Network/virtualNetworks/csels-cdh-dev-vnet/subnets/cdh-azml-dev-snet since it is in use with resource /subscriptions/8cac8a7b-baf4-4965-ba8d-db72ccbb7c68/resourceGroups/223572a6-d321-404f-b766-4ed67aca786e-AzureBatch-CloudService/providers/Microsoft.Net

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 1cdd7ede-42c5-4644-ac57-12f9a39e2522
More information can be found using '.get_logs()'
Error:
{
  "code": "IpConfigProfileCannotReferToSubnetInUseWithResource",
  "statusCode": 400,
  "message": "Vnet Service request failed. Reason: IpConfigurationProfile /subscriptions/320d8d57-c87c-4434-827f-59ee7d86687a/resourceGroups/csels-cdh-dev/providers/Microsoft.Network/networkProfiles/aci-network-profile-csels-cdh-dev-vnet-cdh-azml-dev-snet/containerNetworkInterfaceConfigurations/eth0/ipConfigurations/ipconfig1 cannot refer to subnet /subscriptions/320d8d57-c87c-4434-827f-59ee7d86687a/resourceGroups/CSELS-CDH-DEV/providers/Microsoft.Network/virtualNetworks/csels-cdh-dev-vnet/subnets/cdh-azml-dev-snet since it is in use with resource /subscriptions/8cac8a7b-baf4-4965-ba8d-db72ccbb7c68/resourceGroups/223572a6-d321-404f-b766-4ed67aca

In [ ]:
print(service.get_logs())

In [ ]:
import json
import numpy as np
import pandas as pd 

input_sample = np.array([[0.393845668409139, -0.7457496952627328, -0.22189398515428446, -0.7676382304726264, -0.6999246324662508, -0.17745016145311038, 
-0.81484548808207, -0.770581761525873, -0.7189314051409194, 0.07367558407930598, -0.4665409990624931, -0.06425072325003647, -0.667435753669589, 
-0.10209930505592255, -0.2928749021935234, -0.18393920631817678, -0.8204798568998558, -0.6065568523529427, -0.395651155530143, 0.3134950272756633,
 -0.8678658791041564, -0.6897420117050609, -0.5598110994362666, -0.723009965205552, -0.6542518616646612, -0.5936861839456196,
 -0.9574791012332506, -0.9270503937797329, -0.7552651134179409, -0.2651797994882257, -1.0640128515011344]])

input_payload = json.dumps({
    'data': input_sample.tolist()
})

output = service.run(input_payload)

print(output)

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "data": [
    [
      0.393845668409139,
      -0.7457496952627328,
      -0.22189398515428446,
      -0.7676382304726264,
      -0.6999246324662508,
      -0.17745016145311038,
      -0.81484548808207,
      -0.770581761525873,
      -0.7189314051409194,
      0.07367558407930598,
      -0.4665409990624931,
      -0.06425072325003647,
      -0.667435753669589,
      -0.10209930505592255,
      -0.2928749021935234,
      -0.18393920631817678,
      -0.8204798568998558,
      -0.6065568523529427,
      -0.395651155530143,
      0.3134950272756633,
      -0.8678658791041564,
      -0.6897420117050609,
      -0.5598110994362666,
      -0.723009965205552,
      -0.6542518616646612,
      -0.5936861839456196,
      -0.9574791012332506,
      -0.9270503937797329,
      -0.7552651134179409,
      -0.2651797994882257,
      -1.0640128515011344
    ]
  ]
}

body = str.encode(json.dumps(data))

url = 'http://d29e1866-d35b-4f23-8827-ea1dd1bffd9a.eastus2.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

## AKS deployment

In [50]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-secure-workspace-vnet?tabs=pe%2Ccli

from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

aks_target = AksCompute(ws,"aks-cdh-dev")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1)
service = Model.deploy(ws, 
                        service_name, 
                        [model], 
                        inference_config, 
                        deployment_config, 
                        aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-09-06 18:14:46+00:00 Creating Container Registry if not exists.
2022-09-06 18:14:46+00:00 Registering the environment.
2022-09-06 18:14:47+00:00 Use the existing image.
2022-09-06 18:14:48+00:00 Creating resources in AKS.
2022-09-06 18:14:49+00:00 Submitting deployment to compute.
2022-09-06 18:14:49+00:00 Checking the status of deployment breast-cancer-custom-service.
TimedOut


Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 7777de4a-c33f-4dd0-8386-039caf994256
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "Timed out waiting for AKS deployment to complete. pollTimeout : 00:20:00 serviceName: breast-cancer-custom-service serviceId: 16339ae6-e93b-404f-87df-917b87bd5cc7",
  "details": [
    {
      "code": "DeploymentTimedOut",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. F

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 7777de4a-c33f-4dd0-8386-039caf994256
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "Timed out waiting for AKS deployment to complete. pollTimeout : 00:20:00 serviceName: breast-cancer-custom-service serviceId: 16339ae6-e93b-404f-87df-917b87bd5cc7",
  "details": [
    {
      "code": "DeploymentTimedOut",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-